In [3]:
# Dependencies
import pandas as pd
import requests
import json

import os
import csv
import datetime

from csv import writer
from csv import reader

# Hide warning messages in notebook
import warnings
warnings.filterwarnings('ignore')

# pretty print the output
from pprint import pprint 

In [8]:
store_list_df = pd.read_csv('<input.csv>')
store_list_df.head()

In [9]:
with open('<input.csv>') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            print(f'\t{row[0]}, {row[1]} ,{row[2]},{row[3]},{row[4]},{row[5]}')
            line_count += 1
    print(f'Processed {line_count} lines.')


In [10]:
with open('<input.csv>') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            print(f'\t{row[0]}, {row[1]} ,{row[2]},{row[3]},{row[4]}')
            base_url = "https://api.weather.com/v3/wx/hod/conditions/historical/point?pointType=nearest&geocode="+row[2]+","+row[3]+"&startDateTime="+row[4]+"&endDateTime="+row[5]+"&units=e&format=csv&apiKey=<YOUR API KEY>"
            file_name = row[0]+row[1]+"_"+row[4]+"_"+row[5]+"_weather_data.csv"

            #Build the query url     
            query_url = base_url
    
            try:
               # Get weather data
                weather_response = requests.get(url=query_url,
                                        headers={'Content-Type': 'application/octet-stream'}
                                       )
                print("Weather_response",weather_response)
                #check for the response 200 then proceed,
                if weather_response is not None and weather_response.status_code == 200:
                    with open(file_name, 'ab') as csr:
                        for chunk in weather_response.iter_content(chunk_size=1024):
                            if chunk:
                                csr.write(chunk)

            except (KeyError, IndexError):
                print("Missing field/result... skipping.")
                print("----------")
                line_count += 1
    print(f'Processed {line_count} lines.')

In [11]:
# List all files in a directory using os.listdir
basepath = '<YOUR DIRECTORY PATH>'
for entry in os.listdir(basepath):
    if os.path.isfile(os.path.join(basepath, entry)):
        #Create a directory with out_2021
        with open(os.path.join(basepath, entry), 'r') as read_obj, \
                open('out_2021/weather_2021.csv', 'a', newline='') as write_obj:
        # Create a csv.reader object from the input file object
            csv_reader = reader(read_obj)
            next(csv_reader, None)
        # Create a csv.writer object from the output file object
            csv_writer = writer(write_obj)
        # Read each row of the input csv file as list
            for row in csv_reader:
                # To select the ROWS with T23:00 hour(Select particular row)
                if 'T23:' in row[3]:
                    # Append the default text in the row / list
                    if len(entry) == 51:
                        row.append(entry[3:8])
                        csv_writer.writerow(row)
                    elif len(entry) == 50:
                        row.append(entry[2:7])
                        csv_writer.writerow(row)
                    else:
                        csv_writer.writerow(row)
            print('end of file')